In [1]:
import pandas as pd

all=pd.read_csv("imdb.csv",encoding='unicode_escape')
all=all.loc[all['label']!="unsup"]
X_test=(all.loc[all['type']=='test']).loc[:,['review']]
y_test=(all.loc[all['type']=='test']).loc[:,['label']]

X_train=(all.loc[all['type']=='train']).loc[:,['review']]
y_train=(all.loc[all['type']=='train']).loc[:,['label']]


In [2]:
#tokenizing reviews
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~`'''
test_reviews=X_test.to_dict()
train_reviews=X_train.to_dict()

#print(test_reviews['review'])
def tokenizing(rev):
 tokens = nltk.word_tokenize(rev)
 stop_words = set(stopwords.words('english'))
 tokens_l = [w.lower() for w in tokens]
 token_final=[]
 for item in tokens_l : 
  item= item.strip(punc)
  if item not in stop_words and item !="":
   token_final.append(item)
 text = nltk.Text(tokens)
 final_text=""
 for wd in token_final:
  final_text=final_text+wd+" " 
 #only_nn = [x for (x,y) in token_final if y in ('NN')]
 #match = text.concordance('language')
 return [final_text, token_final]

actual_test_text=dict()
for x in test_reviews['review']:
 #print(x)
 #print(test_reviews['review'][x])
 actual_review=test_reviews['review'][x]
 txt=tokenizing(actual_review)[0]
 actual_test_text[x]=txt

actual_train_text=dict()
for x in train_reviews['review']:
 #print(x)
 #print(test_reviews['review'][x])
 actual_review=train_reviews['review'][x]
 txt=tokenizing(actual_review)[0]
 actual_train_text[x]=txt

 #print(actual_test)

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_test["label"])
y_test=le.transform(y_test["label"])

le.fit(y_train["label"])
y_train=le.transform(y_train["label"])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer# Create feature vectors
df_test_text=pd.DataFrame.from_dict(actual_test_text, orient="index", columns=["review"])
df_train_text=pd.DataFrame.from_dict(actual_train_text, orient="index", columns=["review"])
#print(df_test_text.all)

vectorizer = TfidfVectorizer(max_features=2000,
                             min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
x_train_vectors = vectorizer.fit_transform(df_train_text['review']).toarray()
x_test_vectors = vectorizer.transform(df_test_text['review']).toarray()


In [6]:
print(x_train_vectors)
print(y_train)
print(x_test_vectors)
print(y_test)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.06155073 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[0 0 0 ... 1 1 1]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.07950486 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[0 0 0 ... 1 1 1]


In [7]:
from sklearn import svm

clf = svm.SVC(kernel='linear', C=0.1)
clf.fit(x_train_vectors, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pred=clf.predict(x_test_vectors)
print(confusion_matrix(y_test,pred))  
print(classification_report(y_test,pred))  
print(accuracy_score(y_test, pred))

[[    0     0 12500]
 [    0     0 12500]
 [    0     0     0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00   12500.0
           1       0.00      0.00      0.00   12500.0
           2       0.00      0.00      0.00       0.0

    accuracy                           0.00   25000.0
   macro avg       0.00      0.00      0.00   25000.0
weighted avg       0.00      0.00      0.00   25000.0

0.0


/home/jonh/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonh/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonh/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonh/.local/lib/python3.9/site-packages/sklearn/